In [2]:
import requests
import json, ndjson
from collections import Counter

with open("secret.csv", "r") as r:
    data = [line.replace("\n","").split(",") for line in r.readlines()]
    username = data[0][1]
    token = data[1][1]

In [3]:
url = "https://lichess.org"
print(f"Gathering data for {username}")
response = requests.get(
  f'https://lichess.org/api/games/user/{username}',
  params={
    "perfType" : "blitz",
    "rated" :  "true",
    "opening" : "true",
    "moves" : "true"
  },
  headers={
    'Authorization': f'Bearer {token}', # Need this or you will get a 401: Not Authorized response
    "Accept": "application/x-ndjson"
  }
)

Gathering data for sirsnakerb


In [4]:
# Parse application/x-ndjson into list of JSON objects
games = []
ndjson = response.content.decode().split('\n')

for json_obj in ndjson:
    if json_obj:
        games.append(json.loads(json_obj))

In [5]:
len(games)

3700

In [6]:
filtered_games = []
for game in games:
    if game.get("winner"):
        unw = game.get("players").get("white").get("user").get("name")
        unb = game.get("players").get("black").get("user").get("name")
        if (game.get("winner") == "white" and username == unw) or (game.get("winner") == "black" and username == unb):
            conclusion = "won"
        else:
            conclusion = "lost"
    else:
        conclusion = "tie"

    filtered_game = {
        "id" : game["id"],
        "moves" : game["moves"].split(" "),
        "outcome" : conclusion
    } 
    filtered_games.append(filtered_game) 

In [9]:
from functools import reduce  
import operator

def get_by_path(root, items):
    """Access a nested object in root by item sequence."""
    return reduce(operator.getitem, items, root)

def set_by_path(root, items, value):
    """Set a value in a nested object in root by item sequence."""
    get_by_path(root, items[:-1])[items[-1]] = value

In [18]:
depth = 3
tree = {}

for i in range(1, depth):
    for game in filtered_games:
        moves = game.get("moves")[:i]
        print(moves)
        if len(moves) == i: #else it will overwrite branches with short games
            set_by_path(tree, moves, {"score" : {
                    "won" : 0,
                    "lost" : 0,
                    "tie" : 0
            }})
            

for i in range(1, depth):
    for game in filtered_games:
        moves = game.get("moves")[:i]
        moves.append("score")
        moves.append(game["outcome"])
        set_by_path(tree, moves, get_by_path(tree, moves) + 1)

['e4']
['e4']
['g3']
['d4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['d4']
['e4']
['c4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['b3']
['e4']
['d4']
['d4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['d4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['d4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['d4']
['e4']
['e4']
['d4']
['e4']
['e4']
['e4']
['e4']
['e3']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['d4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['c4']
['e4']
['d4']
['e3']
['e4']
['e4']
['e4']
['d4']
['e4']
['d4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['c4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['Nf3']
['e4']
['e4']
['d4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['e4']
['f4'

In [17]:
a = {"mt" : {"e4": {"score": {"won": 1005, "lost": 1146, "tie": 120},
  "c5": {"score": {"won": 126, "lost": 145, "tie": 13}},
  "e5": {"score": {"won": 490, "lost": 559, "tie": 64}},
  "d5": {"score": {"won": 180, "lost": 228, "tie": 15}},
  "e6": {"score": {"won": 62, "lost": 48, "tie": 7}},
  "c6": {"score": {"won": 20, "lost": 22, "tie": 3}},
  "Na6": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "b6": {"score": {"won": 9, "lost": 19, "tie": 1}},
  "Nc6": {"score": {"won": 15, "lost": 16, "tie": 3}},
  "b5": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "g6": {"score": {"won": 10, "lost": 13, "tie": 2}},
  "d6": {"score": {"won": 32, "lost": 38, "tie": 6}},
  "a6": {"score": {"won": 2, "lost": 2, "tie": 0}},
  "h6": {"score": {"won": 4, "lost": 0, "tie": 0}},
  "Nf6": {"score": {"won": 39, "lost": 33, "tie": 4}},
  "f5": {"score": {"won": 4, "lost": 3, "tie": 0}},
  "g5": {"score": {"won": 2, "lost": 1, "tie": 0}},
  "a5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "f6": {"score": {"won": 9, "lost": 14, "tie": 2}},
  "h5": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "g3": {"score": {"won": 10, "lost": 15, "tie": 0},
  "d5": {"score": {"won": 5, "lost": 12, "tie": 0}},
  "e5": {"score": {"won": 2, "lost": 1, "tie": 0}},
  "b6": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "f5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "e6": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "Nf6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "d6": {"score": {"won": 1, "lost": 0, "tie": 0}}},
 "d4": {"score": {"won": 386, "lost": 467, "tie": 34},
  "f5": {"score": {"won": 23, "lost": 25, "tie": 2}},
  "d5": {"score": {"won": 224, "lost": 266, "tie": 22}},
  "d6": {"score": {"won": 15, "lost": 22, "tie": 2}},
  "g6": {"score": {"won": 13, "lost": 9, "tie": 3}},
  "Nc6": {"score": {"won": 15, "lost": 21, "tie": 1}},
  "Nf6": {"score": {"won": 25, "lost": 30, "tie": 2}},
  "e6": {"score": {"won": 33, "lost": 49, "tie": 2}},
  "e5": {"score": {"won": 18, "lost": 22, "tie": 0}},
  "c6": {"score": {"won": 4, "lost": 6, "tie": 0}},
  "f6": {"score": {"won": 4, "lost": 7, "tie": 0}},
  "h5": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "b6": {"score": {"won": 6, "lost": 4, "tie": 0}},
  "c5": {"score": {"won": 4, "lost": 1, "tie": 0}},
  "g5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "a5": {"score": {"won": 1, "lost": 1, "tie": 0}},
  "h6": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "c4": {"score": {"won": 31, "lost": 23, "tie": 2},
  "e5": {"score": {"won": 14, "lost": 4, "tie": 0}},
  "f5": {"score": {"won": 3, "lost": 2, "tie": 0}},
  "d5": {"score": {"won": 8, "lost": 6, "tie": 0}},
  "c6": {"score": {"won": 2, "lost": 0, "tie": 0}},
  "Nc6": {"score": {"won": 2, "lost": 1, "tie": 0}},
  "c5": {"score": {"won": 2, "lost": 2, "tie": 0}},
  "e6": {"score": {"won": 0, "lost": 4, "tie": 0}},
  "Nf6": {"score": {"won": 0, "lost": 3, "tie": 1}},
  "d6": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "g6": {"score": {"won": 0, "lost": 0, "tie": 1}}},
 "b3": {"score": {"won": 6, "lost": 6, "tie": 1},
  "e5": {"score": {"won": 0, "lost": 2, "tie": 0}},
  "d5": {"score": {"won": 3, "lost": 2, "tie": 1}},
  "f6": {"score": {"won": 1, "lost": 1, "tie": 0}},
  "g6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "Nf6": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "Nc6": {"score": {"won": 1, "lost": 0, "tie": 0}}},
 "e3": {"score": {"won": 40, "lost": 53, "tie": 3},
  "c5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "d5": {"score": {"won": 15, "lost": 13, "tie": 2}},
  "e5": {"score": {"won": 16, "lost": 22, "tie": 1}},
  "b6": {"score": {"won": 1, "lost": 1, "tie": 0}},
  "Nf6": {"score": {"won": 2, "lost": 3, "tie": 0}},
  "Nc6": {"score": {"won": 1, "lost": 3, "tie": 0}},
  "b5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "c6": {"score": {"won": 1, "lost": 2, "tie": 0}},
  "d6": {"score": {"won": 0, "lost": 4, "tie": 0}},
  "f6": {"score": {"won": 1, "lost": 1, "tie": 0}},
  "e6": {"score": {"won": 1, "lost": 4, "tie": 0}}},
 "Nf3": {"score": {"won": 36, "lost": 33, "tie": 4},
  "d5": {"score": {"won": 23, "lost": 15, "tie": 2}},
  "c5": {"score": {"won": 2, "lost": 1, "tie": 0}},
  "g6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "b6": {"score": {"won": 3, "lost": 0, "tie": 0}},
  "Nf6": {"score": {"won": 1, "lost": 3, "tie": 1}},
  "e5": {"score": {"won": 1, "lost": 2, "tie": 0}},
  "e6": {"score": {"won": 1, "lost": 7, "tie": 0}},
  "f6": {"score": {"won": 0, "lost": 2, "tie": 1}},
  "Nc6": {"score": {"won": 1, "lost": 2, "tie": 0}},
  "d6": {"score": {"won": 2, "lost": 0, "tie": 0}},
  "c6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "f5": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "f4": {"score": {"won": 7, "lost": 12, "tie": 0},
  "d5": {"score": {"won": 4, "lost": 7, "tie": 0}},
  "b6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "f5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "h5": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "e5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "Nf6": {"score": {"won": 0, "lost": 3, "tie": 0}},
  "Nc6": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "a3": {"score": {"won": 1, "lost": 0, "tie": 0},
  "e5": {"score": {"won": 1, "lost": 0, "tie": 0}}},
 "f3": {"score": {"won": 13, "lost": 6, "tie": 4},
  "d5": {"score": {"won": 6, "lost": 2, "tie": 3}},
  "c5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "e5": {"score": {"won": 4, "lost": 4, "tie": 1}},
  "c6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "Nc6": {"score": {"won": 1, "lost": 0, "tie": 0}}},
 "g4": {"score": {"won": 1, "lost": 3, "tie": 0},
  "d5": {"score": {"won": 1, "lost": 2, "tie": 0}},
  "d6": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "Nc3": {"score": {"won": 30, "lost": 44, "tie": 4},
  "g6": {"score": {"won": 1, "lost": 3, "tie": 0}},
  "e5": {"score": {"won": 10, "lost": 22, "tie": 2}},
  "f5": {"score": {"won": 2, "lost": 0, "tie": 0}},
  "c6": {"score": {"won": 2, "lost": 1, "tie": 0}},
  "d5": {"score": {"won": 6, "lost": 7, "tie": 0}},
  "e6": {"score": {"won": 1, "lost": 3, "tie": 0}},
  "Nc6": {"score": {"won": 2, "lost": 3, "tie": 0}},
  "Nf6": {"score": {"won": 4, "lost": 3, "tie": 2}},
  "c5": {"score": {"won": 0, "lost": 2, "tie": 0}},
  "d6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "b6": {"score": {"won": 1, "lost": 0, "tie": 0}}},
 "": {"score": {"won": 6, "lost": 10, "tie": 0}},
 "d3": {"score": {"won": 42, "lost": 50, "tie": 5},
  "d5": {"score": {"won": 10, "lost": 13, "tie": 1}},
  "e5": {"score": {"won": 18, "lost": 22, "tie": 3}},
  "d6": {"score": {"won": 0, "lost": 2, "tie": 0}},
  "Nf6": {"score": {"won": 1, "lost": 1, "tie": 1}},
  "e6": {"score": {"won": 5, "lost": 7, "tie": 0}},
  "b6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "a5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "g6": {"score": {"won": 1, "lost": 2, "tie": 0}},
  "c5": {"score": {"won": 1, "lost": 3, "tie": 0}},
  "Nc6": {"score": {"won": 2, "lost": 0, "tie": 0}},
  "c6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "f5": {"score": {"won": 1, "lost": 0, "tie": 0}}},
 "b4": {"score": {"won": 2, "lost": 6, "tie": 0},
  "d5": {"score": {"won": 1, "lost": 2, "tie": 0}},
  "e5": {"score": {"won": 0, "lost": 3, "tie": 0}},
  "f5": {"score": {"won": 1, "lost": 1, "tie": 0}}},
 "c3": {"score": {"won": 10, "lost": 22, "tie": 1},
  "g6": {"score": {"won": 1, "lost": 1, "tie": 0}},
  "b6": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "Nf6": {"score": {"won": 1, "lost": 1, "tie": 0}},
  "d5": {"score": {"won": 2, "lost": 3, "tie": 0}},
  "e5": {"score": {"won": 3, "lost": 15, "tie": 1}},
  "e6": {"score": {"won": 1, "lost": 1, "tie": 0}},
  "b5": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "c5": {"score": {"won": 1, "lost": 0, "tie": 0}}},
 "h3": {"score": {"won": 1, "lost": 1, "tie": 0},
  "e5": {"score": {"won": 1, "lost": 0, "tie": 0}},
  "e6": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "h6": {"score": {"won": 2, "lost": 0, "tie": 0}},
 "Nh3": {"score": {"won": 0, "lost": 2, "tie": 0},
  "e5": {"score": {"won": 0, "lost": 2, "tie": 0}}},
 "Nc6": {"score": {"won": 0, "lost": 1, "tie": 0},
  "Qg4": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "Bg5": {"score": {"won": 0, "lost": 1, "tie": 0},
  "e5": {"score": {"won": 0, "lost": 1, "tie": 0}}},
 "e6": {"score": {"won": 0, "lost": 0, "tie": 1},
  "cxd5": {"score": {"won": 0, "lost": 0, "tie": 1}}},
 "h4": {"score": {"won": 0, "lost": 2, "tie": 0},
  "d5": {"score": {"won": 0, "lost": 1, "tie": 0}},
  "Nf6": {"score": {"won": 0, "lost": 1, "tie": 0}}}}}

In [18]:
b = a["mt"]["e4"]

In [19]:
b

{'score': {'won': 1005, 'lost': 1146, 'tie': 120},
 'c5': {'score': {'won': 126, 'lost': 145, 'tie': 13}},
 'e5': {'score': {'won': 490, 'lost': 559, 'tie': 64}},
 'd5': {'score': {'won': 180, 'lost': 228, 'tie': 15}},
 'e6': {'score': {'won': 62, 'lost': 48, 'tie': 7}},
 'c6': {'score': {'won': 20, 'lost': 22, 'tie': 3}},
 'Na6': {'score': {'won': 0, 'lost': 1, 'tie': 0}},
 'b6': {'score': {'won': 9, 'lost': 19, 'tie': 1}},
 'Nc6': {'score': {'won': 15, 'lost': 16, 'tie': 3}},
 'b5': {'score': {'won': 0, 'lost': 1, 'tie': 0}},
 'g6': {'score': {'won': 10, 'lost': 13, 'tie': 2}},
 'd6': {'score': {'won': 32, 'lost': 38, 'tie': 6}},
 'a6': {'score': {'won': 2, 'lost': 2, 'tie': 0}},
 'h6': {'score': {'won': 4, 'lost': 0, 'tie': 0}},
 'Nf6': {'score': {'won': 39, 'lost': 33, 'tie': 4}},
 'f5': {'score': {'won': 4, 'lost': 3, 'tie': 0}},
 'g5': {'score': {'won': 2, 'lost': 1, 'tie': 0}},
 'a5': {'score': {'won': 1, 'lost': 0, 'tie': 0}},
 'f6': {'score': {'won': 9, 'lost': 14, 'tie': 2}},

In [20]:
def to_keep_one(dictionary, item):
    """Keeps one key in the dict"""
    keep = dictionary[item]
    dictionary.clear() 
    dictionary[item] = keep
    return dictionary

In [21]:
to_keep_one(b, "score")

{'score': {'won': 1005, 'lost': 1146, 'tie': 120}}